In [1]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
from transformers import BertTokenizerFast, BertForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_PATH = "/content/drive/MyDrive/emotion_model"

tokenizer = BertTokenizerFast.from_pretrained(MODEL_PATH)
model = BertForSequenceClassification.from_pretrained(MODEL_PATH)

model.to(device)
model.eval()




BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [6]:
from google.colab import files
uploaded = files.upload()


Saving Testing_sentence35000.xlsx to Testing_sentence35000.xlsx


In [7]:
import pandas as pd

df_test = pd.read_excel("Testing_sentence.xlsx")
df_test.head()


,sentence
0,i feel sort of disturbed we unearthed somethin...
1,i feel really sad that my own girlfriend canno...
2,i asked him feeling curious because i m unable...
3,i been feeling a lot more emotional
4,i feel like a selfish fuck for writing about t...


In [8]:
label2id = {
    'anger': 0,
    'fear': 1,
    'joy': 2,
    'love': 3,
    'sad': 4,
    'suprise': 5,
    'study/work': 6
}

id2label = {v: k for k, v in label2id.items()}



In [9]:
encodings = tokenizer(
    list(df_test["sentence"]),
    truncation=True,
    padding=True,
    return_tensors="pt"
)

encodings = {k: v.to(device) for k, v in encodings.items()}

with torch.no_grad():
    outputs = model(**encodings)

pred_ids = torch.argmax(outputs.logits, dim=1).cpu().numpy()
pred_labels = [id2label[i] for i in pred_ids]



In [10]:
df_test["predicted_emotion"] = pred_labels

print("\n===== MODEL PREDICTIONS (NO GROUND TRUTH YET) =====\n")
print(df_test[["sentence", "predicted_emotion"]].head(20))



===== MODEL PREDICTIONS (NO GROUND TRUTH YET) =====

                                             sentence predicted_emotion
0   i feel sort of disturbed we unearthed somethin...               sad
1   i feel really sad that my own girlfriend canno...               sad
2   i asked him feeling curious because i m unable...           suprise
3                 i been feeling a lot more emotional               sad
4   i feel like a selfish fuck for writing about t...             anger
5   i feel like i m constantly scrounging to put s...               sad
6   i managed a whole tuesday of eating clean but ...              fear
7     i feel she was being generous in her assessment              love
8   i feel like most of these songs came pretty ea...               joy
9   i landed safely at the tiny airport here feeli...           suprise
10  i am improving my skills through regular job p...        study/work
11      i went downstairs feeling depressed and angry               sad
12  i dont

In [11]:
df_test.to_excel("predictions_only.xlsx", index=False)


In [12]:
#PHASE 2 — UPLOAD ANSWERS & EVALUATE

uploaded = files.upload()


Saving Testing_emotion35000.xlsx to Testing_emotion35000.xlsx


In [14]:
df_true = pd.read_excel("Testing_emotion35000.xlsx")
df_true.head()


,emotion
0,sad
1,sad
2,suprise
3,sad
4,anger


In [15]:
df_test["actual_emotion"] = df_true["emotion"]
df_test["correct"] = df_test["predicted_emotion"] == df_test["actual_emotion"]


In [16]:
print("\n===== PREDICTED vs ACTUAL =====\n")
print(df_test[["sentence", "predicted_emotion", "actual_emotion", "correct"]].head(20))




===== PREDICTED vs ACTUAL =====

                                             sentence predicted_emotion  \
0   i feel sort of disturbed we unearthed somethin...               sad   
1   i feel really sad that my own girlfriend canno...               sad   
2   i asked him feeling curious because i m unable...           suprise   
3                 i been feeling a lot more emotional               sad   
4   i feel like a selfish fuck for writing about t...             anger   
5   i feel like i m constantly scrounging to put s...               sad   
6   i managed a whole tuesday of eating clean but ...              fear   
7     i feel she was being generous in her assessment              love   
8   i feel like most of these songs came pretty ea...               joy   
9   i landed safely at the tiny airport here feeli...           suprise   
10  i am improving my skills through regular job p...        study/work   
11      i went downstairs feeling depressed and angry             

In [17]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(
    df_test["actual_emotion"],
    df_test["predicted_emotion"]
)

print("\n===== ACCURACY =====")
print("Accuracy:", accuracy * 100, "%")



===== ACCURACY =====
Accuracy: 93.0 %


In [18]:
print("\n===== CLASSIFICATION REPORT =====\n")
print(classification_report(
    df_test["actual_emotion"],
    df_test["predicted_emotion"]
))



===== CLASSIFICATION REPORT =====

              precision    recall  f1-score   support

       anger       0.95      0.98      0.97       155
        fear       0.94      0.92      0.93       140
         joy       0.81      0.92      0.86       154
        love       0.96      1.00      0.98       152
         sad       0.99      0.92      0.96       128
  study/work       1.00      0.74      0.85       127
     suprise       0.92      1.00      0.96       144

    accuracy                           0.93      1000
   macro avg       0.94      0.93      0.93      1000
weighted avg       0.94      0.93      0.93      1000



In [19]:
df_test.to_excel("prediction_vs_actual.xlsx", index=False)
files.download("prediction_vs_actual.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
df_test["correct"].value_counts()


,count
correct,
True,930
False,70


In [21]:
df_test[df_test["correct"] == False][
    ["sentence", "predicted_emotion", "actual_emotion"]
].head(10)


,sentence,predicted_emotion,actual_emotion
7,i feel she was being generous in her assessment,love,joy
40,i feel confident about progressing in my thesi...,joy,study/work
62,i feel challenged but determined in my interns...,joy,study/work
72,i read this quote and i think about the things...,love,joy
86,i feel challenged but determined in my interns...,joy,study/work
93,i preached to a f students in a gospel camp th...,anger,joy
99,my sister gave birth to twins,sad,joy
106,i love this blazer the fit and feel are amazing,suprise,joy
109,i feel challenged but determined in my interns...,joy,study/work
112,i have burned the candle for about hours and b...,anger,sad
